In [3]:
import utils
from Core.get_pRef import get_pRef_from_metaheuristic
from BenchmarkProblems.RoyalRoad import RoyalRoad

problem = RoyalRoad(5)

pRef = get_pRef_from_metaheuristic(problem=problem,
                                   sample_size=10000,
                                   which_algorithm="GA",
                                   unique=True)

print(pRef)

PRef with 926 samples, mean = 7.97


In [68]:


from typing import Optional
import numpy as np
import itertools
from Core.PS import PS, STAR
from Core.PRef import PRef


def PRef_to_PS_list(pRef: PRef):
    return [(PS.from_FS(efs), (efs.fitness,)) for efs in pRef.get_evaluated_FSs()]

def get_maybe_merged(ps_a, ps_b, fits_a, fits_b, max_genome_diff, max_fitness_diff) -> Optional:
    diff_count = sum(a != b for a, b in zip(ps_a.values, ps_b.values) if a != STAR and b != STAR)
    if diff_count > max_genome_diff:
        return None
    merged_fits = tuple(set(fits_a + fits_b))
    max_diff = max(merged_fits) - min(merged_fits)
    if max_diff > max_fitness_diff:
        return None
    
    merged_PS = PS(a if a==b else STAR for a, b in zip(ps_a.values, ps_b.values))
    return merged_PS, merged_fits    

def get_dict_counts(d):
    return (len(d.keys()), sum(len(val) for val in d.values()))


def get_list_counts(d):
    return len(d), sum(len(val) for _, val in d)
        
def merge_list(ps_list: list[(PS, set[float])], max_genome_diff: int, max_fitness_diff: float):
    verbose = False # len(ps_list) < 30
    def log(msg):
        if verbose:
            print(msg)
    
    merged_list = []
    merged_set = set()
    unmerged_list = []
    for index_a, pair_a in enumerate(ps_list):
        ps_a, fits_a = pair_a
        log(f"Considering {ps_a}, {fits_a}")
        if pair_a in merged_set:
            log(f"{ps_a} was skipped because it's in the merged set")
            continue
        for ps_b, fits_b in ps_list[(index_a+1):]:
            if (ps_b, fits_b) in merged_set:
                log(f"{ps_b}, {fits_b} was not considered by A because it's in the merged_set")
                continue
            was_merged = get_maybe_merged(ps_a, ps_b, fits_a, fits_b, max_genome_diff, max_fitness_diff)
            if was_merged:
                log(f"{ps_b}, {fits_b} got merged! {was_merged}")
                merged_list.append(was_merged)
                
                merged_set.add(pair_a)
                merged_set.add((ps_b, fits_b))
                break
        
        if pair_a not in merged_set:
            log(f"At the end of its cycle, a was not added anywhere, so it gets added to the unmerged list")
            unmerged_list.append(pair_a)
            
    if False:
        print("Merged:")
        for item, fit_set in merged_list:
            print(item, fit_set)
        print("Unmerged:")
        for item, fit_set in unmerged_list:
            print(item, fit_set)
    
    result_list = merged_list + unmerged_list
    if verbose:
        print(f"original = {get_list_counts(ps_list)}, "
              f"merged = {get_list_counts(merged_list)}, "
              f"unmerged = {get_list_counts(unmerged_list)},"
              f"result = {get_list_counts(result_list)}")
    return result_list
    
    
def merge_until_static(ps_list, max_genome_diff: int, max_fitness_diff: float):
    current = ps_list
    while True:
        new_list = merge_list(current, max_genome_diff, max_fitness_diff)
        if len(current) == len(new_list):
            return current
        current = new_list


In [70]:
initial_ps_list = PRef_to_PS_list(pRef)

merged = merge_until_static(initial_ps_list, max_genome_diff=1, max_fitness_diff=0.5)

for ps, values in sorted(merged, key = lambda pair: max(pair[1])):
    print(ps, values)


[* 0 0 1 1 0 1 1 0 1 0 0 1 1 0 0 0 0 * 0] (0.0,)
[1 1 0 0 1 0 1 * 1 1 0 0 0 0 1 0 0 1 1 0] (0.0,)
[0 1 0 1 0 * 1 1 0 1 0 0 0 1 1 1 0 0 1 1] (0.0,)
[1 1 0 1 0 0 0 0 * 0 0 1 1 0 0 0 0 0 1 1] (0.0,)
[0 0 0 1 0 1 1 0 1 0 1 0 0 0 * 1 0 1 0 0] (0.0,)
[1 1 0 0 0 * 0 0 1 1 1 0 0 1 1 1 1 0 1 0] (0.0,)
[0 0 1 1 1 1 0 1 1 * 0 1 0 1 1 1 1 0 1 1] (0.0,)
[0 0 0 1 0 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0] (0.0,)
[1 1 1 0 1 0 1 0 0 1 1 1 1 0 0 1 0 0 0 1] (0.0,)
[0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0 1 0 0] (0.0,)
[0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0] (0.0,)
[1 0 1 1 1 1 1 0 0 0 1 0 0 1 0 1 0 1 0 1] (0.0,)
[1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 0 0 0] (0.0,)
[0 0 1 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0] (0.0,)
[1 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0] (0.0,)
[0 0 0 1 1 1 0 0 0 1 1 0 1 0 1 0 0 0 0 0] (0.0,)
[0 1 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 1 1 0] (0.0,)
[0 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1 0 0] (0.0,)
[1 0 0 1 0 1 0 0 0 0 1 1 1 0 0 1 1 0 1 1] (0.0,)
[1 0 1 1 0 0 1 0 1 1 0 0 0 0 1 1 0 1 0 1] (0.0,)
[1 0 1 1 1 1 1 0 1 0

In [39]:
print(np.sum(pRef.fitness_array == 0))

146
